In [1]:
# import dependencies 
import pickle 
from tqdm.notebook import tqdm
import numpy as np
from allennlp.data.tokenizers.spacy_tokenizer import SpacyTokenizer
import plotnine
from plotnine import *
import nltk
import pandas as pd
import os 
from utils import *
from minimality import *
import spacy
import matplotlib.pyplot as plt
plt.style.use('bmh')
import warnings
warnings.filterwarnings('ignore')

In [4]:
polyjuice = load_pickle("C:\\Users\\acer\Documents\edits_from_aris\pickle_files\\newsgroups_polyjuice_random.pkl")
polyjuice_news_adj = load_pickle("C:\\Users\\acer\Documents\edits_from_aris\pickle_files\\newsgroups_polyjuice_adj.pkl")
polyjuice_news_nouns = load_pickle("C:\\Users\\acer\Documents\edits_from_aris\pickle_files\\newsgroups_polyjuice_noun.pkl")
polyjuice_news_verbs = load_pickle("C:\\Users\\acer\Documents\edits_from_aris\pickle_files\\newsgroups_polyjuice_verb.pkl")

#### For imdb

In [28]:
polyjuice = load_pickle("C:\\Users\\acer\Documents\edits_from_aris\pickle_files\imdb_polyjuice.pickle")
polyjuice_news_adj = load_pickle("C:\\Users\\acer\Documents\edits_from_aris\pickle_files\imdb_polyjuice_random_adj.pkl")
polyjuice_news_nouns = load_pickle("C:\\Users\\acer\Documents\edits_from_aris\pickle_files\imdb_polyjuice_random_noun.pkl")
polyjuice_news_verbs = load_pickle("C:\\Users\\acer\Documents\edits_from_aris\pickle_files\imdb_polyjuice_random_verb.pkl")

In [29]:
indexes = [polyjuice.index(item1) for item1, item2, item3, item4 in zip(polyjuice, polyjuice_news_adj, polyjuice_news_nouns, polyjuice_news_verbs) if len(item1) > 1 and len(item2) > 1 and len(item3)>1 and len(item4)>1]
polyjuice = [polyjuice[i] for i in indexes]
polyjuice_news_adj = [polyjuice_news_adj[i] for i in indexes]
polyjuice_news_nouns = [polyjuice_news_nouns[i] for i in indexes]
polyjuice_news_verbs = [polyjuice_news_verbs[i] for i in indexes]

In [30]:
import re

def replace_extra_spaces(text):
    # Replace extra spaces with html space notation "&nbsp;
    replaced_text = re.sub(" ", r"<u>&nbsp;</u>", text)

    return replaced_text

In [31]:
import difflib
from allennlp.data.tokenizers import Token
from allennlp.data.tokenizers.spacy_tokenizer import SpacyTokenizer

def html_highlight_diffs(orig, edited, tokenizer_wrapper=SpacyTokenizer()):
    """ Given an orig and edited inputs, mark up differences in HTML. """

    orig = orig.replace("<br ", "<-br ")
    edited = edited.replace("<br ", "<-br ")

    orig_tok = tokenizer_wrapper.tokenize(orig)
    edited_tok = tokenizer_wrapper.tokenize(edited)

    orig_text_tok = [t.text for t in orig_tok]
    edited_text_tok = [t.text for t in edited_tok]

    edited_mark_indices, num_add, num_del = get_marked_indices(orig_text_tok, 
                                                        edited_text_tok, "+")
    orig_mark_indices, num_add_2, num_del_2 = get_marked_indices(orig_text_tok, 
                                                        edited_text_tok, "-")
    marked_original = orig 
    for idx in reversed(orig_mark_indices):
        token = orig_tok[idx]
        start, end = token.idx, token.idx_end
        if start == None or end == None:
            logger.info(token, start, end)
        marked_original = marked_original[:start] + "<b>" + \
                marked_original[start:end] + "</b>" + marked_original[end:]

    marked_edited = edited.replace("<br />", "<-br />") 
    for idx in reversed(edited_mark_indices):
        token = edited_tok[idx]
        start, end = token.idx, token.idx_end
        if start == None or end == None:
            logger.info(token, start, end)
        marked_edited = marked_edited[:start] + "<b>" + \
                marked_edited[start:end] + "</b>" + marked_edited[end:]

    # Replace extra spaces with HTML notation for spaces
    marked_original = replace_extra_spaces(marked_original)
    marked_edited = replace_extra_spaces(marked_edited)
    
    return marked_original, marked_edited

def get_marked_indices(orig_tokinal, tokenized_contrast, symbol):
    """ Helper function for html_highlight_diffs. 
    Will only return indices of words deleted or replaced (not inserted). """

    index_offset = 0
    d = difflib.Differ()
    diff = d.compare(orig_tokinal, tokenized_contrast)
    list_diff = list(diff)
    tokens, modified_tokens, indices = [], [], []
    counter = 0
    additions, deletions = 0, 0

    for token_idx, token in enumerate(list_diff):
        marker = token[0]
        word = token[2:]
        if marker == symbol:        
            tokens.append(word)
            indices.append(counter)
            counter += 1
        elif marker == " ":
            modified_tokens.append(word)
            counter += 1

        if marker == "+":
            additions += 1
        if marker == "-":
            deletions += 1
            
    return indices, additions, deletions

In [32]:
def highlight_diffs_list(input, j):
    """ Given a list of sentences, mark up differences in HTML. """
    if len(input) > j+1:
        sentences = [item[0] for item in input[j]]
        edited_sentences = [html_highlight_diffs(sentences[i], sentences[i+1]) for i in range(0, len(input[j])-1)]
        edited_sentences = [edited_sentences[0][0]] + [tup[1] for tup in edited_sentences]    # Remove identical sentences
    else:
        sentences = [item[0] for item in input[j]]
        edited_sentences = [html_highlight_diffs(sentences[i], sentences[i+1]) for i in range(0, len(input[j])-1)]
        edited_sentences = [edited_sentences[0][0]] + [tup[1] for tup in edited_sentences]
    return edited_sentences

def get_minims(input, j):
    if len(input) > j+1:
        sentences = [item[0] for item in input[j]]
        minims = [str(score_minimality(sentences[i], sentences[i+1])) for i in range(0, len(input[j])-1)]
    else:
        sentences = [item[0] for item in input[j]]
        minims = [str(score_minimality(sentences[i], sentences[i+1])) for i in range(0, len(input[j])-1)]
    return minims

In [33]:
polyjuice1 = sorted(polyjuice,key=lambda x: len(x[0][0]))
polyjuice_adj1 = sorted(polyjuice_news_adj,key=lambda x: len(x[0][0]))
polyjuice_nouns1 = sorted(polyjuice_news_nouns,key=lambda x: len(x[0][0]))
polyjuice_verbs1 = sorted(polyjuice_news_verbs,key=lambda x: len(x[0][0]))

In [34]:
from IPython.display import HTML

output = "<table><tr><th style=\"text-align: left\">{}</th><th style=\"text-align: left\">{}</th><th style=\"text-align: left\">{}</th><th style=\"text-align: left\">{}</th><th style=\"text-align: left\">{}</th><th style=\"text-align: left\">{}</th><th style=\"text-align: left\">{}</th><th style=\"text-align: left\">{}</th></tr>".format("<b>Minimalities Polyjuice</b>", "<b>Minimalities Polyjuice Adj</b>","<b>Minimalities Polyjuice Noun</b>", "<b>Minimalities Polyjuice Verb</b>", "<b>Polyjuice</b>", "<b>Polyjuice Adj</b>", "<b>Polyjuice Noun</b>", "<b>Polyjuice Verb</b>")
for i in range(30):
    output += "<tr><td style=\"text-align: left\">{}</td><td style=\"text-align: left\">{}</td><td style=\"text-align: left\">{}</td><td style=\"text-align: left\">{}</td><td style=\"text-align: left\">{}</td><td style=\"text-align: left\">{}</td><td style=\"text-align: left\">{}</td><td style=\"text-align: left\">{}</td>".format("<br>".join(get_minims(polyjuice1, i)), "<br>".join(get_minims(polyjuice_adj1, i)), "<br>".join(get_minims(polyjuice_nouns1, i)), "<br>".join(get_minims(polyjuice_verbs1, i)), "<br>".join(highlight_diffs_list(polyjuice1, i)), "<br>".join(highlight_diffs_list(polyjuice_adj1, i)), "<br>".join(highlight_diffs_list(polyjuice_nouns1, i)), "<br>".join(highlight_diffs_list(polyjuice_verbs1, i)))

output += "</table>"

display(HTML(output))